# Classifier Training

In [1]:
! rsync -a /kaggle/input/mmdetection-v280/mmdetection /
! pip install /kaggle/input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3/
! pip install /kaggle/input/hpapytorchzoo/pytorch_zoo-master/
! pip install /kaggle/input/hpacellsegmentation/HPA-Cell-Segmentation/
! pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

! cp -r /kaggle/input/kgl-humanprotein-data/kgl_humanprotein_data /
! cp -r /kaggle/input/humanpro/kgl_humanprotein /

import sys
sys.path.append('/kgl_humanprotein/')

Processing /kaggle/input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3
  Created wheel for mmpycocotools: filename=mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl size=272913 sha256=943a126d36dc3959955bb4098233948690f5e6bb12594406b6ba72646e0b2886
  Stored in directory: /root/.cache/pip/wheels/80/e0/da/3288fdf3965b5c9090f368462db9d28be2c82013f51821090a
Successfully built mmpycocotools
Processing /kaggle/input/hpapytorchzoo/pytorch_zoo-master
  Created wheel for pytorch-zoo: filename=pytorch_zoo-0.0.0-py3-none-any.whl size=30139 sha256=c7e686d9821dd08b4dea1550c3cfdb89ae9136caaa2fe7f8a4dff1a55adc3807
  Stored in directory: /root/.cache/pip/wheels/7f/18/21/aff5a8914e22461b2b025a9629c2b70464c36183caaf12bc09
Successfully built pytorch-zoo
Processing /kaggle/input/hpacellsegmentation/HPA-Cell-Segmentation
  Created wheel for hpacellseg: filename=hpacellseg-0.1.8-py3-none-any.whl size=14815 sha256=bd75fcca45e9e0db8da4a576a66dca7f1432b0f23676d28238794a421c8c439f
  Stored in di

In [2]:
import os
import time
from pathlib import Path
import shutil
import zipfile
import functools
import multiprocessing
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import KFold,StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import torch
from torch.backends import cudnn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import DataParallel
import matplotlib.pyplot as plt
from tqdm import tqdm

from kgl_humanprotein.utils.common_util import *
from kgl_humanprotein.config.config import *
from kgl_humanprotein.data_process import *
from kgl_humanprotein.datasets.tool import image_to_tensor
from kgl_humanprotein.networks.imageclsnet import init_network
from kgl_humanprotein.layers.loss import *
from kgl_humanprotein.layers.scheduler import *
from kgl_humanprotein.utils.augment_util import train_multi_augment2
from kgl_humanprotein.utils.log_util import Logger
from kgl_humanprotein.run.train import *

run on 88d3eac0af0b


In [3]:
%cd /kaggle

/kaggle


## Combine subsets' meta data

In [4]:
dir_data = Path('/kaggle/input')
dir_mdata = Path('/kaggle/mdata')

In [5]:
%%time
# df_cells = combine_subsets_metadata(dir_data, n_subsets)
df_cells = pd.read_feather('/kaggle/input/humanpro-data-multilabel-cells-meta/train.feather')

CPU times: user 826 ms, sys: 468 ms, total: 1.29 s
Wall time: 4.02 s


In [6]:
dir_mdata_raw = dir_mdata/'raw'
dir_mdata_raw.mkdir(exist_ok=True, parents=True)

df_cells.to_feather(dir_mdata_raw/'train.feather')

In [7]:
del df_cells

## Filter samples

In [8]:
# Remove samples whose target is ''
df_cells = pd.read_feather(dir_mdata_raw/'train.feather')
df_cells = df_cells[df_cells['Target'] != ''].reset_index(drop=True)

In [9]:
df_cells.shape

(153899, 9)

In [10]:
# Limit number of samples per label

def cap_number_per_label(df_cells, cap=10_000, idx_start=0):
    df_cells_cap = pd.DataFrame()
    for label in df_cells.Target.unique():
        df = df_cells[df_cells.Target==label]
        if len(df) > cap:
            df = df.iloc[idx_start:idx_start + cap]
        df_cells_cap = df_cells_cap.append(df, ignore_index=True)
    return df_cells_cap

In [11]:
df_cells = cap_number_per_label(df_cells, cap=5_000, idx_start=0) 

In [12]:
df_cells.Target.value_counts()

16         5000
13         5000
5          5000
0          5000
14         5000
           ... 
9|17          1
9|13|16       1
0|2|13        1
0|1|2         1
0|5|12        1
Name: Target, Length: 149, dtype: int64

In [13]:
df_cells.to_feather(dir_mdata_raw/'train.feather')

## One-hot encode labels

In [14]:
%%time
generate_meta(dir_mdata, 'train.feather')

CPU times: user 22.7 s, sys: 364 ms, total: 23 s
Wall time: 23 s


## Split generation

In [15]:
%%time
train_meta = pd.read_feather(dir_mdata/'meta'/'train_meta.feather')
create_random_split(dir_mdata, train_meta, n_splits=5, alias='random')
del train_meta

Nucleoplasm 7922 1981
Nuclear membrane 4720 1180
Nucleoli 5069 1267
Nucleoli fibrillar center 3050 762
Nuclear speckles 4136 1034
Nuclear bodies 5091 1272
Endoplasmic reticulum 1601 400
Golgi apparatus 4715 1178
Intermediate filaments 2322 580
Actin filaments 3698 924
Microtubules 4177 1044
Mitotic spindle 0 0
Centrosome 3283 821
Plasma membrane 5771 1443
Mitochondria 4513 1128
Aggresome 488 121
Cytosol 6082 1521
Vesicles and punctate cytosolic patterns 805 202
Negative 0 0
create split file: /kaggle/mdata/split/random_folds5/random_train_cv0.feather, shape: (60311, 29)
create split file: /kaggle/mdata/split/random_folds5/random_valid_cv0.feather, shape: (15082, 29)
create split file: /kaggle/mdata/split/random_folds5/random_train_cv1.feather, shape: (60334, 29)
create split file: /kaggle/mdata/split/random_folds5/random_valid_cv1.feather, shape: (15059, 29)
create split file: /kaggle/mdata/split/random_folds5/random_train_cv2.feather, shape: (60368, 29)
create split file: /kaggle/mdat

## Training

In [16]:
from kgl_humanprotein.datasets.protein_dataset import ProteinDataset
from torch.utils.data.sampler import WeightedRandomSampler

def main_training(dir_data, dir_mdata, dir_results, out_dir, gpu_id='0', 
                  arch='class_densenet121_dropout', pretrained=None, model_multicell=None, 
                  num_classes=19, in_channels=4, loss='FocalSymmetricLovaszHardLogLoss',
                  scheduler='Adam45', epochs=55, img_size=768, crop_size=512, batch_size=32, 
                  workers=3, pin_memory=True, split_name='random_ext_folds5', fold=0, 
                  clipnorm=1, resume=None):
    '''
    PyTorch Protein Classification.  Main training function.
    
    Args:
        dir_data (str, Path): Directory where training subsets are.
        dir_mdata (str, Path): Directory where training meta data is.
        dir_results (std, Path): Directory to save training results.
        out_dir (str): 
            Name/label for this training run.  Will be used to create
            directory under `dir_results`.
        gpu_id (str): GPU id used for training. Default: '0'
        arch (str): Model architecture.  
            Default: ``'class_densenet121_dropout'``
        pretrained (Path, str): Path to a pretrained model.  These are
            the parameters just before training starts.
        model_multicell (Path, str): Path to multi-cell model 
            to start training from. Default: None
        num_classes (int): Number of classes. Default: 19 
        in_channels (int): In channels. Default: 4
        loss (str, optional): Loss function. 
            One of ``'FocalSymmetricLovaszHardLogLoss'``. 
            Default: ``'FocalSymmetricLovaszHardLogLoss'``
        scheduler (str): Scheduler name. Default: ``'Adam45'``
        epochs (int): Number of total epochs to run. Default: 55
        img_size (int): Image size.  Default: 768
        crop_size (int): Crop size.  Default: 512
        batch_size (int): Train mini-batch size. Default: 32
        workers (int): Number of data loading workers. Default: 3
        pin_memory (bool): DataLoader's ``pin_memory`` argument.
        split_name (str, optional): Split name.  
            One of: ``'random_ext_folds5'``, 
            or ``'random_ext_noleak_clean_folds5'``. 
            Default: ``'random_ext_folds5'``
        fold (int): Index of fold. Default: 0
        clipnorm (int): Clip grad norm'. Default: 1
        resume (str): Name of the latest checkpoint. Default: None
    '''
    log_out_dir = opj(dir_results, 'logs', out_dir, 'fold%d' % fold)
    if not ope(log_out_dir):
        os.makedirs(log_out_dir)
    log = Logger()
    log.open(opj(log_out_dir, 'log.train.txt'), mode='a')

    model_out_dir = opj(dir_results, 'models', out_dir, 'fold%d' % fold)
    log.write(">> Creating directory if it does not exist:\n>> '{}'\n".format(model_out_dir))
    if not ope(model_out_dir):
        os.makedirs(model_out_dir)

    # set cuda visible device
    os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
    cudnn.benchmark = True

    # set random seeds
    torch.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)

    model_params = {}
    model_params['architecture'] = arch
    model_params['num_classes'] = num_classes
    model_params['in_channels'] = in_channels
    model = init_network(model_params, model_multicell=model_multicell)
    
    if pretrained:
        print(f'Loading pretrained model {pretrained}')
        checkpoint = torch.load(pretrained)
        model.load_state_dict(checkpoint['state_dict'])
    
    # move network to gpu
    model = DataParallel(model)
    model.to(DEVICE)

    # define loss function (criterion)
    try:
        criterion = eval(loss)().to(DEVICE)
    except:
        raise(RuntimeError("Loss {} not available!".format(loss)))

    start_epoch = 0
    best_loss = 1e5
    best_epoch = 0
    best_focal = 1e5

    # define scheduler
    try:
        scheduler = eval(scheduler)()
    except:
        raise (RuntimeError("Scheduler {} not available!".format(scheduler)))
    optimizer = scheduler.schedule(model, start_epoch, epochs)[0]

    # optionally resume from a checkpoint
    if resume:
        resume = os.path.join(model_out_dir, resume)
        if os.path.isfile(resume):
            # load checkpoint weights and update model and optimizer
            log.write(">> Loading checkpoint:\n>> '{}'\n".format(resume))

            checkpoint = torch.load(resume)
            start_epoch = checkpoint['epoch']
            best_epoch = checkpoint['best_epoch']
            best_focal = checkpoint['best_score']
            model.module.load_state_dict(checkpoint['state_dict'])

            optimizer_fpath = resume.replace('.pth', '_optim.pth')
            if ope(optimizer_fpath):
                log.write(">> Loading checkpoint:\n>> '{}'\n".format(optimizer_fpath))
                optimizer.load_state_dict(torch.load(optimizer_fpath)['optimizer'])
            log.write(">>>> loaded checkpoint:\n>>>> '{}' (epoch {})\n".format(resume, checkpoint['epoch']))
        else:
            log.write(">> No checkpoint found at '{}'\n".format(resume))

    # Data loading code
    train_transform = train_multi_augment2
    train_split_file = dir_mdata / 'split'/ split_name / f'random_train_cv{fold}.feather'
    train_dataset = ProteinDataset(
        dir_data,
        train_split_file,
        img_size=img_size,
        is_trainset=True,
        return_label=True,
        in_channels=in_channels,
        transform=train_transform,
        crop_size=crop_size,
        random_crop=True)
    
    label_weights = get_label_weights(train_dataset.split_df)
    weights = torch.from_numpy(train_dataset.split_df['Target']
                               .apply(lambda o: label_weights[o]).values)
    sampler = WeightedRandomSampler(weights, len(weights))
    
    train_loader = DataLoader(
        train_dataset,
        sampler=sampler,
        batch_size=batch_size,
        drop_last=True,
        num_workers=workers,
        pin_memory=pin_memory)
    
    valid_split_file = (dir_mdata/'split'/split_name/
                        f'random_valid_cv{fold}.feather')
    
    valid_dataset = ProteinDataset(
        dir_data,
        valid_split_file,
        img_size=img_size,
        is_trainset=True,
        return_label=True,
        in_channels=in_channels,
        transform=None,
        crop_size=crop_size,
        random_crop=False)
    
    valid_loader = DataLoader(
        valid_dataset,
        sampler=SequentialSampler(valid_dataset),
        batch_size=batch_size,
        drop_last=False,
        num_workers=workers,
        pin_memory=pin_memory)

    focal_loss = FocalLoss().to(DEVICE)
    log.write('** start training here! **\n')
    log.write('\n')
    log.write('epoch    iter      rate     |  train_loss/acc  |    valid_loss/acc/focal/kaggle     |best_epoch/best_focal|  min \n')
    log.write('-----------------------------------------------------------------------------------------------------------------\n')
    start_epoch += 1
    for epoch in range(start_epoch, epochs + 1):
        end = time.time()

        # set manual seeds per epoch
        np.random.seed(epoch)
        torch.manual_seed(epoch)
        torch.cuda.manual_seed_all(epoch)

        # adjust learning rate for each epoch
        lr_list = scheduler.step(model, epoch, epochs)
        lr = lr_list[0]

        # train for one epoch on train set
        iter, train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, clipnorm=clipnorm, lr=lr)

        with torch.no_grad():
            valid_loss, valid_acc, valid_focal_loss, kaggle_score = validate(valid_loader, model, criterion, epoch, focal_loss)

        # remember best loss and save checkpoint
        is_best = valid_focal_loss < best_focal
        best_loss = min(valid_focal_loss, best_loss)
        best_epoch = epoch if is_best else best_epoch
        best_focal = valid_focal_loss if is_best else best_focal

        print('\r', end='', flush=True)
        log.write('%5.1f   %5d    %0.6f   |  %0.4f  %0.4f  |    %0.4f  %6.4f %6.4f %6.4f    |  %6.1f    %6.4f   | %3.1f min \n' % \
                  (epoch, iter + 1, lr, train_loss, train_acc, valid_loss, valid_acc, valid_focal_loss, kaggle_score,
                   best_epoch, best_focal, (time.time() - end) / 60))

        save_model(model, is_best, model_out_dir, optimizer=optimizer, epoch=epoch, best_epoch=best_epoch, best_focal=best_focal)


In [17]:
model_multicell = (
    '../../kgl_humanprotein_data/result/models/'
    'external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds/'
    'fold0/final.pth')
pretrained = Path(
    '/kaggle/input/humanpro-classifier-crop/results/models/'
    'external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/'
    'fold0/final.pth')

gpu_id = '0' # '0,1,2,3'
arch = 'class_densenet121_dropout'
num_classes = len(LABEL_NAME_LIST)
scheduler = 'Adam55'
epochs = 10 #55
resume = None 
sz_img = 384
crop_size = 256 #512
batch_size = 76
split_name = 'random_folds5'
fold = 0
workers = 3
pin_memory = True

dir_results = Path('results')
dir_results.mkdir(exist_ok=True, parents=True)

out_dir = Path(f'external_crop{crop_size}_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds')

In [18]:
main_training(dir_data, dir_mdata, dir_results, out_dir, 
              split_name=split_name, fold=fold,
              arch=arch, pretrained=pretrained, model_multicell=model_multicell, scheduler=scheduler,
              epochs=epochs, resume=resume,
              img_size=sz_img, crop_size=crop_size, batch_size=batch_size, 
              gpu_id=gpu_id, workers=workers, pin_memory=pin_memory)

>> Creating directory if it does not exist:
>> 'results/models/external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0'
>> Using pre-trained model.
>> Loading multi-cell model.
Loading pretrained model /kaggle/input/humanpro-classifier-crop/results/models/external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/final.pth
** start training here! **

epoch    iter      rate     |  train_loss/acc  |    valid_loss/acc/focal/kaggle     |best_epoch/best_focal|  min 
-----------------------------------------------------------------------------------------------------------------
  1.0     793    0.000300   |  0.8967  0.9768  |    0.9166  0.9740 0.6288 0.7102    |     1.0    0.6288   | 21.5 min 
  2.0     793    0.000300   |  0.4538  0.9833  |    0.8936  0.9745 0.6168 0.7112    |     2.0    0.6168   | 16.7 min 
  3.0     793    0.000300   |  0.3191  0.9865  |    0.7608  0.9758 0.5260 0.7111    |     3.0    0.5260   | 16.3 min 
  4.0     

In [19]:
! cp -r results/ /kaggle/working/.